In [1]:
#boto3 => Python library for calling up AWS services
import boto3,os
import sagemaker
from sagemaker import get_execution_role
import pandas as pd
import numpy as np
role = get_execution_role

In [2]:
#provide the name and location of the files to be stored in the S3 bucket
bucket_name = 'sagemaker-chidu'
train_file_name = 'Train_final.csv'
val_file_name = 'Validation_final.csv'
test_file_name = 'Test_final.csv'

model_output_location = r's3://{0}/LoanPrediction/model'.format(bucket_name)
train_file_location = r's3://{0}/{1}'.format(bucket_name, train_file_name)
val_file_location = r's3://{0}/{1}'.format(bucket_name, val_file_name)
test_file_location = r's3://{0}/{1}'.format(bucket_name, test_file_name)

In [3]:
#splitting the train file into train and validation.

df=pd.read_csv("Train_file.csv")
train,val = np.split(df,[ int(len(df)*0.8)])


In [4]:
train.to_csv("Train_data_file.csv",index=False)
val.to_csv("Validation_file.csv",index=False)

In [5]:
#brinigng the traget column as the first column
def col_to_first(file_name):
    df=pd.read_csv(file_name)
    cols=list(df.columns)
    cols=[cols[-1]]+cols[:-1]
    df=df[cols]
    df["Loan_Status"]=df["Loan_Status"].apply(lambda x:0 if x=="Y" else 1)
    df.to_csv(file_name,index=False)

In [6]:
col_to_first("Train_data_file.csv")
col_to_first("Validation_file.csv")
# col_to_first("Test_file.csv")


In [7]:
#define a method for writing into s3 bucket
def write_to_s3(filename, bucket, key):
    with open(filename, 'rb') as f:
        return boto3.Session().resource('s3').Bucket(bucket).Object(key).upload_fileobj(f)

In [8]:
write_to_s3('Train_data_file.csv', bucket_name, train_file_name)
write_to_s3('Validation_file.csv', bucket_name, val_file_name)
write_to_s3('Test_file.csv', bucket_name, test_file_name)

other method of transfering data from sagemaker local to s3


boto3.Session().resource('s3').Bucket(bucket_name).Object(('Train_file.csv')).upload_file('Train_file.csv')

if respective folder is reqd then
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join('Train',Train_file.csv')).upload_file('Train_file.csv')


In [9]:
from sagemaker import image_uris
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput

In [10]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":'binary:logistic',
        "num_round":"50"}

In [11]:
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
region="us-east-1"
xgboost_container = sagemaker.image_uris.retrieve("xgboost", region, "1.2-2")

In [12]:
bucket_name = 'sagemaker-chidu'
model_output_location = r's3://{0}/LoanPrediction/model'.format(bucket_name)

In [13]:
# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=xgboost_container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1, 
                                          instance_type='ml.m5.2xlarge', 
                                          volume_size=5, # 5 GB 
                                          output_path=model_output_location)

In [14]:
# define the data type and paths to the training and validation datasets
content_type = "csv"
# training_file = sagemaker.session.s3_input(s3_data=train_file_location, content_type =content_type)
train_input = TrainingInput("s3://{0}/{1}".format(bucket_name, train_file_name), content_type=content_type)
validation_input = TrainingInput("s3://{0}/{1}".format(bucket_name, val_file_name), content_type=content_type)

In [15]:
# execute the XGBoost training job
estimator.fit({'train': train_input, 'validation': validation_input})

2021-10-22 09:03:27 Starting - Starting the training job...
2021-10-22 09:03:52 Starting - Launching requested ML instancesProfilerReport-1634893407: InProgress
......
2021-10-22 09:04:52 Starting - Preparing the instances for training.........
2021-10-22 09:06:13 Downloading - Downloading input data
2021-10-22 09:06:13 Training - Downloading the training image...
2021-10-22 09:06:54 Uploading - Uploading generated training model[2021-10-22 09:06:50.480 ip-10-0-156-142.ec2.internal:1 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-10-22:09:06:50:INFO] Imported framework sagemaker_xgboost_container.training
[2021-10-22:09:06:50:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2021-10-22:09:06:50:INFO] No GPUs detected (normal if no gpus installed)
[2021-10-22:09:06:50:INFO] Running XGBoost Sagemaker in algorithm mode
[2021-10-22:09:06:50:INFO] Determined delimiter of CSV input is ','
[2021-10-22:09:06:50:INFO] Determin